### 下載所需資源

In [1]:
!pip install requests pandas bs4 selenium lxml webdriver_manager
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from lxml import etree
import time

### 初始版本

In [2]:
import xml.etree.ElementTree as ET
import requests
import pandas as pd
import re
import json

# RSS Feed URL
feed_url = "https://feeds.feedburner.com/ettoday/realtime"

# 發出請求並獲取 RSS Feed 數據
response = requests.get(feed_url)
root = ET.fromstring(response.content)

# 初始化列表來存儲新聞數據
news_items = []

# 解析 RSS Feed 中的每個項目
for item in root.findall('.//item'):
    title = item.find('title').text.strip().replace('\n', '').replace('\t', '')
    pub_date = item.find('pubDate').text.strip()
    link = item.find('link').text.strip()
    
    # 使用正則表達式清洗標題
    title = re.sub(r'\s+', ' ', title)
    
    news_items.append({'標題': title, '發布日期': pub_date, '新聞連結': link})

# 使用 pandas 創建 DataFrame
df_news = pd.DataFrame(news_items)

# 將資料轉換成 dict
news_dict = df_news.to_dict(orient='records')

# 將 DataFrame 輸出成 CSV 檔案
csv_file_path = 'news.csv'
df_news.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# 將 dict 輸出成 JSON 檔案
json_file_path = 'news.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(news_dict, json_file, ensure_ascii=False, indent=4)

print("Data has been saved to CSV and JSON files.")


                                     標題                             發布日期  \
0     陳華穿彩虹熊聯名T自嘲「脾氣火爆」！被問新戀情只求「幫我拍好看點」  Sat,08 Jun 2024 02:46:00  +0800   
1            19歲青年溺水狂喊救我 「拍攝者不救」還問：你累了嗎  Sat,08 Jun 2024 02:43:00  +0800   
2                輝達研發中心傳落腳南港 潤泰新：無法回應傳聞  Sat,08 Jun 2024 02:12:00  +0800   
3              龍華科大舞獅戰鼓團 賀「毛焦點」進駐桃園青創基地  Sat,08 Jun 2024 02:10:00  +0800   
4       「夜光龍」降臨壽山巖觀音寺 桃園閩南文化節國際龍獅競賽9日登場  Sat,08 Jun 2024 02:09:00  +0800   
5               社造進駐南崁高中 培力「學生播客」發現在地故事  Sat,08 Jun 2024 02:06:00  +0800   
6                 桃園虛擬運動委員會啟動 議長邱奕勝接任主委  Sat,08 Jun 2024 02:05:00  +0800   
7           推廣黑毛豬料理！ 首屆桃園「脆皮燒肉節」6／10起報名  Sat,08 Jun 2024 02:04:00  +0800   
8            暨大開設微型創業提案課 讓學生爬上樹頂學習創業家精神  Sat,08 Jun 2024 02:02:00  +0800   
9               特斯拉中國上線百度地圖 5月銷量逆勢暴增77%  Sat,08 Jun 2024 01:54:00  +0800   
10           基隆潮境公園山崩！ 東北角端午連假「替代道路一次看」  Sat,08 Jun 2024 01:52:00  +0800   
11     妹子闖非洲工作「2年賺224萬」 吐辛酸血淚：勇敢的人先享受世界  Sat,08 Jun 2024 01:47:00  +0800   
12       中國海

### 寫成class的版本

In [1]:
import xml.etree.ElementTree as ET
import requests
import pandas as pd
import re
import json

class ETTodayCrawler:
    def __init__(self, feed_url):
        self.feed_url = feed_url
        self.news_items = []

    def fetch_feed(self):
        response = requests.get(self.feed_url)
        if response.status_code == 200:
            root = ET.fromstring(response.content)
            self.parse_feed(root)
        else:
            print(f"無法訪問 RSS Feed：{self.feed_url}")

    def parse_feed(self, root):
        for item in root.findall('.//item'):
            title = item.find('title').text.strip().replace('\n', '').replace('\t', '')
            pub_date = item.find('pubDate').text.strip()
            link = item.find('link').text.strip()
            
            # 使用正則表達式清洗標題
            title = re.sub(r'\s+', ' ', title)
            
            self.news_items.append({'標題': title, '發布日期': pub_date, '新聞連結': link})

    def to_dataframe(self):
        return pd.DataFrame(self.news_items)

    def to_dict(self):
        return self.to_dataframe().to_dict(orient='records')

    def save_to_files(self, csv_path, json_path):
        df_news = self.to_dataframe()
        
        # 將 DataFrame 輸出成 CSV 檔案
        df_news.to_csv(csv_path, index=False, encoding='utf-8-sig')
        
        # 將 dict 輸出成 JSON 檔案
        with open(json_path, 'w', encoding='utf-8') as json_file:
            json.dump(self.to_dict(), json_file, ensure_ascii=False, indent=4)

# 使用範例
crawler = ETTodayCrawler(feed_url="https://feeds.feedburner.com/ettoday/realtime")
crawler.fetch_feed()

# 轉換為 DataFrame
df_news = crawler.to_dataframe()
print(df_news)

# 存檔
crawler.save_to_files('news.csv', 'news.json')
print("Data has been saved to CSV and JSON files.")


                                     標題                             發布日期  \
0     陳華穿彩虹熊聯名T自嘲「脾氣火爆」！被問新戀情只求「幫我拍好看點」  Sat,08 Jun 2024 02:46:00  +0800   
1            19歲青年溺水狂喊救我 「拍攝者不救」還問：你累了嗎  Sat,08 Jun 2024 02:43:00  +0800   
2                輝達研發中心傳落腳南港 潤泰新：無法回應傳聞  Sat,08 Jun 2024 02:12:00  +0800   
3              龍華科大舞獅戰鼓團 賀「毛焦點」進駐桃園青創基地  Sat,08 Jun 2024 02:10:00  +0800   
4       「夜光龍」降臨壽山巖觀音寺 桃園閩南文化節國際龍獅競賽9日登場  Sat,08 Jun 2024 02:09:00  +0800   
5               社造進駐南崁高中 培力「學生播客」發現在地故事  Sat,08 Jun 2024 02:06:00  +0800   
6                 桃園虛擬運動委員會啟動 議長邱奕勝接任主委  Sat,08 Jun 2024 02:05:00  +0800   
7           推廣黑毛豬料理！ 首屆桃園「脆皮燒肉節」6／10起報名  Sat,08 Jun 2024 02:04:00  +0800   
8            暨大開設微型創業提案課 讓學生爬上樹頂學習創業家精神  Sat,08 Jun 2024 02:02:00  +0800   
9               特斯拉中國上線百度地圖 5月銷量逆勢暴增77%  Sat,08 Jun 2024 01:54:00  +0800   
10           基隆潮境公園山崩！ 東北角端午連假「替代道路一次看」  Sat,08 Jun 2024 01:52:00  +0800   
11     妹子闖非洲工作「2年賺224萬」 吐辛酸血淚：勇敢的人先享受世界  Sat,08 Jun 2024 01:47:00  +0800   
12       中國海